In [37]:
import argparse
import sys
import pandas as pd
import os
sys.path.append(os.path.abspath(".."))

import torch
import numpy as np
from configs import settings
from core_model.custom_model import ClassifierWrapper, load_custom_model
from core_model.dataset import get_dataset_loader
from core_model.train_test import model_forward

set_noise_ratio='0.5'
set_noise_type='asymmetric'
set_uni_name='Coteaching,Coteachingplus,Decoupling'
set_dataset='cifar-10'
set_model='cifar-resnet18'
set_model_suffix='restore'
set_batch_size=64



def execute():
    case = settings.get_case(set_noise_ratio,set_noise_type)
    print('得到的case',case)
    uni_names = set_uni_name
    uni_names = [uni_names] if uni_names is None else uni_names.split(",")
    num_classes = settings.num_classes_dict[set_dataset]

    loaded_model = load_custom_model(set_model, num_classes, load_pretrained=False)
    model = ClassifierWrapper(loaded_model, num_classes)

    _, _, test_loader = get_dataset_loader(
        set_dataset,
        "test",
        None,
        batch_size=set_batch_size,
        shuffle=False,
    )
    _, _, noisy_loader = get_dataset_loader(
        set_dataset,
        "train_noisy",
        case,
        batch_size=set_batch_size,
        shuffle=False,
        label_name="train_noisy_true_label"
    )
    
    results_data=[]

    for uni_name in uni_names:
        print(f"Evaluating {uni_name}:")
        model_ckpt_path = settings.get_ckpt_path(
            set_dataset,
            case,
            set_model,
            model_suffix=set_model_suffix,
            unique_name=uni_name,
        )
        print(f"Loading model from {model_ckpt_path}")
        checkpoint = torch.load(model_ckpt_path)
        model.load_state_dict(checkpoint, strict=False)
        # print(f"Evaluating test_data:")
        # results, embedding = model_test(test_loader, model)
        # print("Results: %.4f" % results)
        print(f"Evaluating train_noisy_data:")
        n_results, n_embedding = model_test(noisy_loader, model)
        # print("Results: %.4f" % results)
        print("Results: ", n_results)
        dict_temp={'uni_name':uni_name,**n_results}
        results_data.append(dict_temp)
    
    df=pd.DataFrame(results_data)
    return df        



def model_test(data_loader, model, device="cuda"):
    eval_results = {}

    predicts, probs, embedding, labels = model_forward(
        data_loader, model, device, output_embedding=True, output_targets=True
    )

    # global acc
    global_acc = np.mean(predicts == labels)
    eval_results["global"] = global_acc.item()

    # class acc
    label_list = sorted(list(set(labels)))
    for label in label_list:
        cls_index = labels == label
        class_acc = np.mean(predicts[cls_index] == labels[cls_index])
        eval_results["label_" + str(label.item())] = class_acc.item()

    return eval_results, embedding

In [38]:
#把所有模型的结果存到一个dataframe 里面
results=pd.DataFrame()
results=execute()
# results.head()

得到的case nr_0.5_nt_asymmetric_cvpr
Loading /nvme/szh/code/tta-mr/data/cifar-10/gen/test_data.npy
Loading /nvme/szh/code/tta-mr/data/cifar-10/gen/nr_0.5_nt_asymmetric_cvpr/train_noisy_data.npy
Evaluating Coteaching:
Loading model from /nvme/szh/code/tta-mr/ckpt/cifar-10/nr_0.5_nt_asymmetric_cvpr/Coteaching/cifar-resnet18_restore.pth
Evaluating train_noisy_data:


<ipython-input-37-bbcf77932c39>:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_ckpt_path)


Results:  {'global': 0.93824, 'label_0': 0.8938906752411575, 'label_1': 0.9897476340694006, 'label_2': 0.865979381443299, 'label_3': 0.8432304038004751, 'label_4': 0.988198269079465, 'label_5': 0.9153526970954357, 'label_6': 0.9829406986190089, 'label_7': 0.9629331184528606, 'label_8': 0.9850393700787402, 'label_9': 0.9542536115569823}
Evaluating Coteachingplus:
Loading model from /nvme/szh/code/tta-mr/ckpt/cifar-10/nr_0.5_nt_asymmetric_cvpr/Coteachingplus/cifar-resnet18_restore.pth
Evaluating train_noisy_data:
Results:  {'global': 0.94696, 'label_0': 0.9646302250803859, 'label_1': 0.9503154574132492, 'label_2': 0.9191118160190325, 'label_3': 0.9517022961203484, 'label_4': 0.970889063729347, 'label_5': 0.9651452282157676, 'label_6': 0.9593826157595451, 'label_7': 0.8944399677679291, 'label_8': 0.931496062992126, 'label_9': 0.9630818619582665}
Evaluating Decoupling:
Loading model from /nvme/szh/code/tta-mr/ckpt/cifar-10/nr_0.5_nt_asymmetric_cvpr/Decoupling/cifar-resnet18_restore.pth
Eva

In [39]:
results.head()

,uni_name,global,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,Coteaching,0.93824,0.893891,0.989748,0.865979,0.843230,0.988198,0.915353,0.982941,0.962933,0.985039,0.954254
1,Coteachingplus,0.94696,0.964630,0.950315,0.919112,0.951702,0.970889,0.965145,0.959383,0.894440,0.931496,0.963082
2,Decoupling,0.95840,0.970257,0.956625,0.931007,0.963579,0.986625,0.977593,0.970755,0.912168,0.939370,0.976726


In [46]:
# #柱状图

# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as fm
# from math import pi
# import numpy as np

# # Defining method categories
# lnl_methods = ['Coteaching', 'Coteachingplus', 'Decoupling', 'NegativeLearning', 'PENCIL']
# unlearning_methods = ['raw', 'GA', 'FT', 'FT_l1', 'FT_prune', 'retrain', 'retrain_ls', 'retrain_sam']

# baseline_methods = ['pretrain', 'inc_train'] 

# datasets = ['cifar-10_sym', 'cifar-100_asym', 'flower-102_sym', 'pet-37_asym']
# display_name_dataset = {
#     'cifar-10_sym' : 'Cifar-10', 
#     'cifar-100_asym': 'Cifar-100', 
#     'flower-102_sym': 'Flower-102', 
#     'pet-37_asym' : 'Oxford-IIIT Pet'
# }

# display_name_legend = {
#     'pretrain' : 'PreTrain', 
#     'inc_train' : 'IncTrain', 
#     'finetune': 'FineTune', 
#     'Coteaching' : 'CoTe.', 
#     'Coteachingplus': 'CoTe.+', 
#     'Decoupling': 'Decoup.', 
#     'NegativeLearning' : 'NegLn.', 
#     'PENCIL' : 'PENCIL', 
#     'raw': 'Raw', 
#     'GA': 'GA', 
#     'FT': 'FT', 
#     'FT_l1': 'FT-$l_{1}$', 
#     'FT_prune' : 'FT$_p$', 
#     'retrain' : 'ReT', 
#     'retrain_ls' : 'ReT$_L$', 
#     'retrain_sam' : 'ReT$_S$', 
#     'CRUL' : 'CRUL'
# }

# # Creating a bar chart to compare accuracy across methods for each dataset
# fig, axes = plt.subplots(2, 2, figsize=(10, 8))
# axes = axes.flatten()
# width = 0.7  # Increased the width of the bars for better visibility
# tfs = 12 # x/y-ticks font size
# title_size = 18 # Title font size
# lfs = 14 # Legend-font-size

# # Colors for different categories
# baseline_colors = ['lightgrey', 'darkgrey', 'silver']
# lnl_colors = plt.cm.Blues(np.linspace(0.4, 1, 5))
# unlearning_colors = plt.cm.Greens(np.linspace(0.4, 1, 8))
# crul_color = 'red'


# # Plotting each dataset in a separate subplot
# for idx, dataset in enumerate(datasets):
#     ax = axes[idx]
#     x = np.arange(len(baseline_methods) + len(lnl_methods) + len(unlearning_methods) + 1)  # the label locations
    
#     # Plotting baseline methods (pretrain, noise-train, finetune)
#     for i, method in enumerate(baseline_methods):
#         ax.bar(x[i], sheet1_df.loc[dataset, method], width, label=method, color=baseline_colors[i], edgecolor='black', hatch=['//', '..'][i]) # 仅保留 pretrain和inc_train

#     # Plotting LNL methods
#     for i, method in enumerate(lnl_methods):
#         ax.bar(x[len(baseline_methods) + i], sheet1_df.loc[dataset, method], width, label=method, color=lnl_colors[i], edgecolor='black')

#     # Plotting Unlearning methods
#     for i, method in enumerate(unlearning_methods):
#         ax.bar(x[len(baseline_methods) + len(lnl_methods) + i], sheet1_df.loc[dataset, method], width, label=method, color=unlearning_colors[i], edgecolor='black')

#     ax.bar(x[-1], sheet1_df.loc[dataset, 'CRUL'], width, label='CRUL', color=crul_color, edgecolor='black')

#     ax.set_title(f'{display_name_dataset[dataset]}', fontsize=title_size, fontfamily="serif", weight="bold")

#     ax.set_xticks([]) # 关闭x轴显示


# # --------------------------------------------------------------------------------- #
# """
# 手动修正四个子图各自的坐标值范围。
# # """
# # # Cifar-10
# axes[0].set_ylim(40, 85)
# axes[0].set_yticks(np.arange(40, 91, 15))
# axes[0].set_yticklabels(
#             np.arange(40, 91, 15),
#             fontsize=tfs,
#             rotation=90,
#             va="center",
#             fontfamily="serif",
#             weight="bold",
#         )


# # Cifar-100
# axes[1].set_ylim(50, 70)
# axes[1].set_yticks(np.arange(50, 71, 5))
# axes[1].set_yticklabels(
#             np.arange(50, 71, 5),
#             fontsize=tfs,
#             rotation=90,
#             va="center",
#             fontfamily="serif",
#             weight="bold",
#         )

# # Flower-102
# axes[2].set_ylim(50, 95)
# axes[2].set_yticks(np.arange(50, 96, 15))
# axes[2].set_yticklabels(
#             np.arange(50, 96, 15),
#             fontsize=tfs,
#             rotation=90,
#             va="center",
#             fontfamily="serif",
#             weight="bold",
#         )


# # Pet-37
# axes[3].set_ylim(70, 95)
# axes[3].set_yticks(np.arange(70, 96, 10))
# axes[3].set_yticklabels(
#             np.arange(70, 96, 10),
#             fontsize=tfs,
#             rotation=90,
#             va="center",
#             fontfamily="serif",
#             weight="bold",
#         )


# handles, legend_labels = fig.axes[0].get_legend_handles_labels()

# labels = [display_name_legend[ll] for ll in legend_labels]

# legend_font_properties = fm.FontProperties(
#                                         #    weight='bold', 
#                                            size=lfs)

# from matplotlib.lines import Line2D
# empty_handle = Line2D([], [], color='none')  # 创建一个空的占位符

# # 在第二个 legend 之后插入占位符
# handles.insert(2, empty_handle)
# labels.insert(2, '')  # 对应的标签留空

# fig.legend(handles, labels, loc='lower center', 
#            bbox_to_anchor=(0.5, -0.05), ncol=6,
#            prop=legend_font_properties)

# fig.tight_layout(rect=[0, 0.1, 1, 1])

# plt.show()

In [45]:
# #雷达图
# import numpy as np
# import matplotlib.pyplot as plt
# from math import pi

# # CIFAR-10的标签
# labels = ['label0', 'label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9']

# # 假设数据
# our_method_scores = [0.85, 0.88, 0.82, 0.80, 0.83, 0.87, 0.84, 0.86, 0.81, 0.89]
# resnet18_scores = [0.75, 0.77, 0.74, 0.70, 0.73, 0.78, 0.76, 0.75, 0.72, 0.74]

# # 设置雷达图的角度
# num_vars = len(labels)

# # 角度计算（每个标签对应一个角度）
# angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# # 将数据循环到开始位置，这样雷达图会闭合
# our_method_scores += our_method_scores[:1]
# resnet18_scores += resnet18_scores[:1]
# angles += angles[:1]

# # 创建雷达图
# fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# # 画 Our method 和 ResNet18 的雷达图
# ax.plot(angles, our_method_scores, color='blue', linewidth=2, label='Our method')
# ax.fill(angles, our_method_scores, color='blue', alpha=0.25)

# ax.plot(angles, resnet18_scores, color='red', linewidth=2, label='ResNet18')
# ax.fill(angles, resnet18_scores, color='red', alpha=0.25)

# # 设置标签（CIFAR-10的类别）
# ax.set_yticklabels([])
# ax.set_xticks(angles[:-1])
# ax.set_xticklabels(labels, fontsize=10)

# # 添加标题
# ax.set_title('Comparison of Our method and ResNet18 on CIFAR-10', size=14, color='black', y=1.1)

# # 添加图例
# ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.05))

# # 显示图形
# plt.show()

In [44]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 方法名和对应的准确率（根据实际结果替换这些数据）
# methods = ['Our method', 'ResNet18', 'Method A', 'Method B']
# accuracies = [0.85, 0.75, 0.78, 0.80] # 假设这四种方法在 CIFAR-10 上的准确率

# # 设置柱状图的宽度和位置
# x = np.arange(len(methods)) # 方法的数量
# width = 0.5 # 每个柱子的宽度

# # 创建柱状图
# fig, ax = plt.subplots(figsize=(8, 6))

# # 绘制柱状图
# bars = ax.bar(x, accuracies, width, color=['blue', 'red', 'green', 'orange'])

# # 设置标签和标题
# ax.set_xlabel('Methods', fontsize=12)
# ax.set_ylabel('Accuracy', fontsize=12)
# ax.set_title('Comparison of Different Methods on CIFAR-10', fontsize=14)
# ax.set_xticks(x) # 设置 X 轴刻度位置
# ax.set_xticklabels(methods, fontsize=12) # 设置 X 轴标签

# # 添加柱状图的数值标签
# for bar in bars:
#     height = bar.get_height()
#     ax.annotate(f'{height:.2f}', # 显示每个柱的准确率
#     xy=(bar.get_x() + bar.get_width() / 2, height),
#     xytext=(0, 3), # 偏移量
#     textcoords="offset points",
#     ha='center', va='bottom', fontsize=10)

# # 显示图形
# plt.show()